<a href="https://colab.research.google.com/github/bzhao10/cs598_dlh_final_project/blob/main/CS_598_Final_pj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CS 598 Final Project

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U strsimpy

     |████████████████████████████████| 45 kB 1.8 MB/s 


In [ ]:
!pip install fasttext


     |████████████████████████████████| 68 kB 2.6 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3142284 sha256=243df30a66ee597e71e2044d2cc8c88179e925a044445654d4fd65ceb524919e
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import pandas as pd

#notes=pd.read_csv("/content/drive/MyDrive/DLH_final_project_dataset/noteevents.csv")

notes=pd.read_csv("/content/drive/MyDrive/DLH_final_project_dataset/diabete_noteevents.csv")
admin_icd=pd.read_csv("/content/drive/MyDrive/DLH_final_project_dataset/admission_id_diagnose_icd_code.csv")

In [ ]:
notes[notes.HADM_ID==100018]

,HADM_ID,ROW_ID,SUBJECT_ID,CATEGORY,CGID,CHARTDATE,CHARTTIME,DESCRIPTION,ISERROR,STORETIME,TEXT
559,100018,699999,58128,Physician,21365.0,2176-08-31,2176-08-31 06:06:00,Intensivist Note,NaN,2176-08-31 06:06:47,sicu hpi male extensive medical history includ...


In [ ]:
def icd_percentage(admin_icd,s):
  res=0
  for i in range(len(admin_icd)):
    if s in admin_icd.loc[i,'icd_codes']:
      res=res+1
  return res/len(admin_icd)
icd_percentage(admin_icd,'272')

0.4361348695836573

In [ ]:
icd_list=[]
for i in range(len(admin_icd)):
  icd_list=icd_list+admin_icd.loc[i,'icd_codes'].split(',')
print(len(set(icd_list)))#4097

4097


In [ ]:
icd_list=set(icd_list)
icd_list_roll=[]
for i in icd_list:
  icd_list_roll.append(i[0:3])
print(len(set(icd_list_roll)))#780

780


In [ ]:
import time
import numpy as np
import string
from nltk import word_tokenize


# PLEASE USE THE GIVEN FUNCTION NAME, DO NOT CHANGE IT


#input
# corpus_df: a dataframe returned by `load_dataset`
#output
# corpus_processed: a list of articles where each article is broke into a list of words 
def preprocess_dataset(df):    
    ''' Preprocess the text data. And return a list of articles. '''
    corpus_processed = []
    corpus_all=[]
    df.TEXT = df.TEXT.str.replace('\n', '')  # remove newline
    df.TEXT = df.TEXT.str.replace('\r', '')  # carriage returns
    """
    TODO: 1. remove punc;
          2. remove numbers.
          
    HINT: consider using `string.punctuation`, `str.maketrans`, and `str.translate`.
    """
    #df.TEXT=df.TEXT.str.replace(string.punctuation, '')

    table = str.maketrans('', '', string.punctuation)
    df.TEXT = [w.translate(table) for w in df.TEXT]
    table2 = str.maketrans('', '', '1234567890')
    df.TEXT = [w.translate(table2) for w in df.TEXT]    
    df.TEXT = df.TEXT.str.lower()  # convert to lower case
    
    # tokenize
    for note in df.TEXT.values:
        note_tokenized = word_tokenize(note)
        corpus_processed.append(note_tokenized)
        corpus_all=corpus_all+note_tokenized
    return corpus_processed,corpus_all

In [ ]:
#https://stackoverflow.com/questions/26483254/python-pandas-insert-list-into-a-cell
notes.at[0, 'TEXT'] = word_tokenize(notes['TEXT'][0])

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
notes_preprocessed,notes_all=preprocess_dataset(notes)

In [ ]:
tmp=pd.value_counts(notes_all)
word_mistake=tmp[tmp<5].index.tolist()
print(len(word_mistake))

6425


In [ ]:
word_common=tmp[tmp>=5].index.tolist()
print(len(word_common))

3273


In [ ]:
word_mistake_dict=dict.fromkeys(word_mistake)

In [ ]:
from strsimpy.levenshtein import Levenshtein
levenshtein = Levenshtein()
count=0
for i in word_mistake_dict:
  count=count+1
  if (count%100==0):
    print(count)
  dist=100000000
  for j in word_common:
    dist_tmp=levenshtein.distance(i,j)
    if dist_tmp<dist:
      dist=dist_tmp
      word_mistake_dict[i]=j


In [ ]:
# initializing string
test_str = 'geekforgeeks best for geeks'
  
# printing original string
print("The original string is : " + str(test_str))
  
# lookup Dictionary
lookp_dict = {"best" : "good and better", "geeks" : "all CS aspirants"}
  
# one-liner to solve problem
res = " ".join(lookp_dict.get(ele, ele) for ele in test_str.split())
  
# printing result 
print("Replaced Strings : " + str(res)) 

In [ ]:
word_mistake_dict

In [ ]:
import fasttext
from gensim.utils import simple_preprocess

#dataset = pd.read_csv('train.csv')[['Body', 'Y']].rename(columns = {'Body': 'questions', 'Y': 'category'})
#ds = pd.read_csv('valid.csv')[['Body', 'Y']].rename(columns = {'Body': 'questions', 'Y': 'category'})

# NLP Preprocess
notes.loc[:, 'TEXT'] = notes.loc[:, 'TEXT'].apply(lambda x: ' '.join(simple_preprocess(x)))
#ds.iloc[:, 0] = ds.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

# Prefixing each row of the category column with '__label__'
#admin_icd.loc[:, 'icd_codes'] = admin_icd.loc[:, 'icd_codes'].apply(map(lambda x: '__label__' + x, x in ))
#ds.iloc[:, 1] = ds.iloc[:, 1].apply(lambda x: '__label__' + x)

In [ ]:
for i in range(len(notes)):
  if (i%100==0):
    print(i)
  for word, initial in word_mistake_dict.items():
    notes.loc[i,'TEXT'] = " ".join(word_mistake_dict.get(ele, ele) for ele in notes.loc[i,'TEXT'].split())
    
    

0
100
200
300
400
500
600
700
800
900


In [ ]:
for i in range(len(notes_preprocessed)):
  for j in range(len(notes_preprocessed[i])):
    if notes_preprocessed[i][j] in word_mistake_dict:
      notes_preprocessed[i][j]=word_mistake_dict
      


In [ ]:
notes.loc[0,'TEXT']

'am chest pa lat clip clip number radiology reason evaluate for lead position admitting diagnosis pneumonia hospital medical condition year old man with new dual chamber icd via left cephalic vein reason for this examination evaluate for lead position final report history year old male with dual chamber pacing device via left cephalic vein please evaluate lead position comparison chest radiograph from three days prior two views of the chest the left chest pacing device has two intact leads projecting over the expected location of the right atrium and ventricle in apparently satisfactory position right upper lobe opacity likely most likely infectious but possibly neoplastic was better seen on cta three days prior minimal vascular engorgement has not changed mild cardiomegaly is stable impression successful placement of pacing device in the expected location'

In [ ]:
admin_icd.loc[:, 'icd_codes'] = admin_icd.loc[:, 'icd_codes'].apply(lambda x: '__label__' + x)
admin_icd.head()

,HADM_ID,icd_codes
0,100001,"__label____label__25013,25043,25053,25063,2508..."
1,100009,"__label____label__25000,2720,27800,2859,4019,4..."
2,100018,"__label____label__2254,25040,25060,25070,2761,..."
3,100034,"__label____label__25000,2724,4019,412,4139,414..."
4,100038,"__label____label__25080,2720,27650,2859,4019,4..."


In [ ]:
for i in range (len(admin_icd)):
  tmp=''
  for j in admin_icd.loc[i, 'icd_codes'].split(","):
    tmp=tmp+'__label__' + j+' '
  admin_icd.loc[i,'icd_codes']=tmp


In [ ]:
notes.TEXT[0]

'am chest pa lat clip clip number radiology reason evaluate for lead position admitting diagnosis pneumonia hospital medical condition year old man with new dual chamber icd via left hepatic vein reason for this examination evaluate for lead position final report history year old male with dual chamber pacing device via left hepatic vein please evaluate lead position comparison chest radiograph from three days prior two views of the chest the left chest pacing device has two intact leads projecting over the expected location of the right atrium and ventricle in apparently satisfactory position right upper lobe opacity likely most likely infectious but possibly neoplasm was better seen on cta three days prior minimal vascular engorgement has not changed mild cardiomegaly is stable impression successful placement of pacing device in the expected location'

In [ ]:
notes.TEXT[0]

'am chest pa lat clip clip number radiology reason evaluate for lead position admitting diagnosis pneumonia hospital medical condition year old man with new dual chamber icd via left cephalic vein reason for this examination evaluate for lead position final report history year old male with dual chamber pacing device via left cephalic vein please evaluate lead position comparison chest radiograph from three days prior two views of the chest the left chest pacing device has two intact leads projecting over the expected location of the right atrium and ventricle in apparently satisfactory position right upper lobe opacity likely most likely infectious but possibly neoplastic was better seen on cta three days prior minimal vascular engorgement has not changed mild cardiomegaly is stable impression successful placement of pacing device in the expected location'

In [ ]:
tmp=0
for id in notes.icd_HADM_ID:
  if (id not in admin_icd.HADM_ID.tolist()):
    print(id)


In [ ]:
100001 in admin_icd.HADM_ID.tolist()

True

In [ ]:
mix_table=pd.merge(admin_icd,notes,on='HADM_ID')

In [ ]:
mix_table.head()

,HADM_ID,icd_codes,ROW_ID,SUBJECT_ID,CATEGORY,CGID,CHARTDATE,CHARTTIME,DESCRIPTION,ISERROR,STORETIME,TEXT
0,100018,__label__2254 __label__25040 __label__25060 __...,699999,58128,Physician,21365.0,2176-08-31,2176-08-31 06:06:00,Intensivist Note,NaN,2176-08-31 06:06:47,sicu hpi male extensive medical history includ...
1,100141,__label__0380 __label__25000 __label__33829 __...,633959,61215,Physician,20453.0,2173-10-01,2173-10-01 07:12:00,Intensivist Note,NaN,2173-10-01 07:12:16,sicu hpi the pt is ym with history of diabetes...
2,100284,__label__25000 __label__2724 __label__2859 __l...,1030855,32077,Radiology,NaN,2152-10-13,2152-10-13 07:48:00,CHEST (PORTABLE AP),NaN,NaN,last name lf first name lf csurg fa am chest p...
3,100350,__label__03842 __label__25060 __label__2948 __...,1135606,56149,Radiology,NaN,2197-06-08,2197-06-08 03:45:00,CHEST (PORTABLE AP),NaN,NaN,am chest portable ap clip clip number radiolog...
4,100524,__label__25041 __label__28529 __label__32723 _...,943037,12508,Radiology,NaN,2198-02-18,2198-02-18 03:21:00,CHEST (PORTABLE AP),NaN,NaN,am chest portable ap clip clip number radiolog...


In [ ]:
import csv
mix_table[['icd_codes', 'TEXT']].to_csv('train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None,
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " " 
                                        )

In [ ]:
len(notes.HADM_ID.unique())

791

In [ ]:
model = fasttext.train_supervised('train.txt', wordNgrams=5)

In [ ]:
model.test('train.txt')


(1000, 0.672, 0.03683806600153492)